# 特征工程
* [使用Sklearn做单机特征工程](http://www.cnblogs.com/jasonfreak/p/5448385.html)

* [更多机器学习方法与应用参考本人网页](http://github.com/lddyato)

In [1]:
from sklearn.datasets import load_iris

In [2]:
iris = load_iris()
X, y = iris.data, iris.target

In [3]:
X[:5]

array([[ 5.1,  3.5,  1.4,  0.2],
       [ 4.9,  3. ,  1.4,  0.2],
       [ 4.7,  3.2,  1.3,  0.2],
       [ 4.6,  3.1,  1.5,  0.2],
       [ 5. ,  3.6,  1.4,  0.2]])

In [4]:
y[:5]

array([0, 0, 0, 0, 0])

# 数据预处理
通过特征提取，我们能得到未经处理的特征，这时的特征可能有以下问题：

1. 不属于同一量纲：即特征的规格不一样，不能够放在一起比较。无量纲化可以解决这一问题。
2. 信息冗余：对于某些定量特征，其包含的有效信息为区间划分，例如学习成绩，假若只关心“及格”或不“及格”，那么需要将定量的考分，转换成“1”和“0”表示及格和未及格。二值化可以解决这一问题。
3. 定性特征不能直接使用：某些机器学习算法和模型只能接受定量特征的输入，那么需要将定性特征转换为定量特征。最简单的方式是为每一种定性值指定一个定量值，但是这种方式过于灵活，增加了调参的工作。通常使用哑编码的方式将定性特征转换为定量特征：假设有N种定性值，则将这一个特征扩展为N种特征，当原始特征值为第i种定性值时，第i个扩展特征赋值为1，其他扩展特征赋值为0。哑编码的方式相比直接指定的方式，不用增加调参的工作，对于线性模型来说，使用哑编码后的特征可达到非线性的效果。
4. 存在缺失值：缺失值需要补充。
5. 信息利用率低：不同的机器学习算法和模型对数据中信息的利用是不同的，之前提到在线性模型中，使用对定性特征哑编码可以达到非线性的效果。类似地，对定量变量多项式化，或者进行其他的转换，都能达到非线性的效果。

## 无量纲化
无量纲化使不同规格的数据转换到同一规格。常见的无量纲化方法有标准化和区间缩放法。标准化的前提是特征值服从正态分布，标准化后，其转换成标准正态分布。区间缩放法利用了边界值信息，将特征的取值区间缩放到某个特点的范围，例如[0, 1]等。

### 标准化
标准化需要计算特征的均值和标准差，公式表达为：<img src='http://images2015.cnblogs.com/blog/927391/201605/927391-20160502113957732-1062097580.png'>

In [9]:
from sklearn.preprocessing import StandardScaler
#标准化，返回值为标准化后的数据
StandardScaler().fit_transform(X)[:5]

array([[-0.90068117,  1.03205722, -1.3412724 , -1.31297673],
       [-1.14301691, -0.1249576 , -1.3412724 , -1.31297673],
       [-1.38535265,  0.33784833, -1.39813811, -1.31297673],
       [-1.50652052,  0.10644536, -1.2844067 , -1.31297673],
       [-1.02184904,  1.26346019, -1.3412724 , -1.31297673]])

### 区间缩放法
区间缩放法的思路有多种，常见的一种为利用两个最值进行缩放，公式表达为：x = (x-Min)/(Max-Min)

In [11]:
from sklearn.preprocessing import MinMaxScaler
#区间缩放，返回值为缩放到[0, 1]区间的数据
MinMaxScaler().fit_transform(X)[:5]

array([[ 0.22222222,  0.625     ,  0.06779661,  0.04166667],
       [ 0.16666667,  0.41666667,  0.06779661,  0.04166667],
       [ 0.11111111,  0.5       ,  0.05084746,  0.04166667],
       [ 0.08333333,  0.45833333,  0.08474576,  0.04166667],
       [ 0.19444444,  0.66666667,  0.06779661,  0.04166667]])

## 归一化
简单来说，标准化是依照特征矩阵的列处理数据，其通过求z-score的方法，将样本的特征值转换到同一量纲下。归一化是依照特征矩阵的行处理数据，其目的在于样本向量在点乘运算或其他核函数计算相似性时，拥有统一的标准，也就是说都转化为“单位向量”。规则为l2的归一化公式如下：
<img src='http://images2015.cnblogs.com/blog/927391/201607/927391-20160719002904919-1602367496.png'>

In [13]:
from sklearn.preprocessing import Normalizer
#归一化，返回值为归一化后的数据
Normalizer().fit_transform(X)[:5]

array([[ 0.80377277,  0.55160877,  0.22064351,  0.0315205 ],
       [ 0.82813287,  0.50702013,  0.23660939,  0.03380134],
       [ 0.80533308,  0.54831188,  0.2227517 ,  0.03426949],
       [ 0.80003025,  0.53915082,  0.26087943,  0.03478392],
       [ 0.790965  ,  0.5694948 ,  0.2214702 ,  0.0316386 ]])

## 对定量特征二值化
定量特征二值化的核心在于设定一个阈值，大于阈值的赋值为1，小于等于阈值的赋值为0，公式表达如下：
<img src='http://images2015.cnblogs.com/blog/927391/201605/927391-20160502115121216-456946808.png'>

In [16]:
from sklearn.preprocessing import Binarizer
# 二值化，阈值设置为3，返回值为二值化后的数据
Binarizer(threshold=3).fit_transform(X)[:5]

array([[ 1.,  1.,  0.,  0.],
       [ 1.,  0.,  0.,  0.],
       [ 1.,  1.,  0.,  0.],
       [ 1.,  1.,  0.,  0.],
       [ 1.,  1.,  0.,  0.]])

## 对定性特征哑变量
由于IRIS数据集的特征皆为定量特征，故使用其目标值进行哑编码（实际上是不需要的）。对于离散特征，例如，性别：｛男，女｝，可以采用one-hot编码的方式将特征表示为一个m维向量，其中m为特征的取值个数。在one-hot向量中只有一个维度的值为1，其余为0。以“性别”这个特征为例，我们可以用向量 “1，0”表示“男”，向量 “0，1”表示“女”。使用one-hot编码可将离散特征的取值扩展到了欧式空间，便于进行相似度计算。使用preproccessing库的OneHotEncoder类对数据进行one-hot编码的代码如下：

In [17]:
from sklearn.preprocessing import OneHotEncoder
#哑编码，对Iris的目标值，返回值为哑编码后的数据
OneHotEncoder().fit_transform(y.reshape(-1,1))

<150x3 sparse matrix of type '<class 'numpy.float64'>'
	with 150 stored elements in Compressed Sparse Row format>

## 缺失值计算
由于IRIS数据集没有缺失值，故对数据集新增一个样本，4个特征均赋值为NaN，表示数据缺失。在实际应用中，我们得到的数据往往不完整，可以用以下方法进行处理：

1. 最简单直接的方法是删除含有缺失值的数据删，这种做法的缺点是可能会导致信息丢失
2. 通过已有数据计算相应特征的平均数、中位数、众数等来补全缺失值
3. 建立一个模型来“预测”缺失的数据。（KNN, Matrix completion等方法）
4. 引入虚拟变量(dummy variable)来表征是否有缺失，是否有补全

In [20]:
from numpy import vstack, array, nan
from sklearn.preprocessing import Imputer
#缺失值计算，返回值为计算缺失值后的数据
#参数missing_value为缺失值的表示形式，默认为NaN
#参数strategy为缺失值填充方式，默认为mean（均值）
Imputer().fit_transform(vstack((array([nan, nan, nan, nan]),X)))[:5]

array([[ 5.84333333,  3.054     ,  3.75866667,  1.19866667],
       [ 5.1       ,  3.5       ,  1.4       ,  0.2       ],
       [ 4.9       ,  3.        ,  1.4       ,  0.2       ],
       [ 4.7       ,  3.2       ,  1.3       ,  0.2       ],
       [ 4.6       ,  3.1       ,  1.5       ,  0.2       ]])

## 数据变换
常见的数据变换有基于多项式的、基于指数函数的、基于对数函数的。
<img src='http://images2015.cnblogs.com/blog/927391/201605/927391-20160502134944451-270339895.png'>

In [23]:
from sklearn.preprocessing import PolynomialFeatures
#多项式转换
# 参数degree为度，默认值为2
PolynomialFeatures().fit_transform(X)

array([[  1.  ,   5.1 ,   3.5 , ...,   1.96,   0.28,   0.04],
       [  1.  ,   4.9 ,   3.  , ...,   1.96,   0.28,   0.04],
       [  1.  ,   4.7 ,   3.2 , ...,   1.69,   0.26,   0.04],
       ..., 
       [  1.  ,   6.5 ,   3.  , ...,  27.04,  10.4 ,   4.  ],
       [  1.  ,   6.2 ,   3.4 , ...,  29.16,  12.42,   5.29],
       [  1.  ,   5.9 ,   3.  , ...,  26.01,   9.18,   3.24]])

基于单变元函数的数据变换可以使用一个统一的方式完成，使用preproccessing库的FunctionTransformer对数据进行对数函数转换的代码如下：

In [25]:
from numpy import log1p
from sklearn.preprocessing import FunctionTransformer

#自定义转换函数为对数函数的数据变换
#第一个参数是单变元函数
FunctionTransformer(log1p).fit_transform(X)[:5]

array([[ 1.80828877,  1.5040774 ,  0.87546874,  0.18232156],
       [ 1.77495235,  1.38629436,  0.87546874,  0.18232156],
       [ 1.74046617,  1.43508453,  0.83290912,  0.18232156],
       [ 1.7227666 ,  1.41098697,  0.91629073,  0.18232156],
       [ 1.79175947,  1.5260563 ,  0.87546874,  0.18232156]])

## 总结
| 类           | 功能| 说明
| :-------------| :------------| :----------------------------------------| 
| StandardScaler	| 无量纲化	| 标准化，基于特征矩阵的列，将特征值转换至服从标准正态分布| 
| MinMaxScaler	| 无量纲化	| 区间缩放，基于最大最小值，将特征值转换到[0, 1]区间上| 
| Normalizer	| 归一化	| 基于特征矩阵的行，将样本向量转换为“单位向量”| 
| Binarizer	| 二值化	| 基于给定阈值，将定量特征按阈值划分| 
| OneHotEncoder	| 哑编码	| 将定性数据编码为定量数据| 
| Imputer	| 缺失值计算	| 计算缺失值，缺失值可填充为均值等| 
|  PolynomialFeatures	| 多项式数据转换	| 多项式数据转换| 
| FunctionTransformer	| 自定义单元数据转换	| 使用单变元的函数来转换数据| 

# 特征选择
当数据预处理完成后，我们需要选择有意义的特征输入机器学习的算法和模型进行训练。通常来说，从两个方面考虑来选择特征：

1. 特征是否发散：如果一个特征不发散，例如方差接近于0，也就是说样本在这个特征上基本上没有差异，这个特征对于样本的区分并没有什么用。
2. 特征与目标的相关性：这点比较显见，与目标相关性高的特征，应当优选选择。除方差法外，本文介绍的其他方法均从相关性考虑。   

根据特征选择的形式又可以将特征选择方法分为3种：

1. Filter：过滤法，按照发散性或者相关性对各个特征进行评分，设定阈值或者待选择阈值的个数，选择特征。
2. Wrapper：包装法，根据目标函数（通常是预测效果评分），每次选择若干特征，或者排除若干特征。
3. Embedded：嵌入法，先使用某些机器学习的算法和模型进行训练，得到各个特征的权值系数，根据系数从大到小选择特征。类似于Filter方法，但是是通过训练来确定特征的优劣。


## Filter

### 方差选择法
使用方差选择法，先要计算各个特征的方差，然后根据阈值，选择方差大于阈值的特征。使用feature_selection库的VarianceThreshold类来选择特征的代码如下

In [26]:
from sklearn.feature_selection import VarianceThreshold
# 方差选择法，返回值为特征选择后的数据
#参数threshold为方差的阈值
VarianceThreshold(threshold=3).fit_transform(X)[:5]

array([[ 1.4],
       [ 1.4],
       [ 1.3],
       [ 1.5],
       [ 1.4]])

In [27]:
X[:6]

array([[ 5.1,  3.5,  1.4,  0.2],
       [ 4.9,  3. ,  1.4,  0.2],
       [ 4.7,  3.2,  1.3,  0.2],
       [ 4.6,  3.1,  1.5,  0.2],
       [ 5. ,  3.6,  1.4,  0.2],
       [ 5.4,  3.9,  1.7,  0.4]])

### 相关系数法
使用相关系数法，先要计算各个特征对目标值的相关系数以及相关系数的P值。皮尔逊系数只能衡量线性相关性,用feature_selection库的SelectKBest类结合相关系数来选择特征的代码如下：

In [14]:
from sklearn.feature_selection import SelectKBest
import numpy as np
from scipy.stats import pearsonr
#选择K个最好的特征，返回选择特征后的数据
#第一个参数为计算评估特征是否好的函数，该函数输入特征矩阵和目标向量，输出二元组（评分，P值）的数组，数组第i项为第i个特征的评分和P值。在此定义为计算相关系数
# 参数K是选择的特征个数
def multivariate_pearsonr(X, y):
    scores, pvalues = [], []
    for ret in map(lambda x:pearsonr(x, y), X.T):
        scores.append(abs(ret[0]))
        pvalues.append(ret[1])
    return (np.array(scores), np.array(pvalues))

SelectKBest(score_func=multivariate_pearsonr, k=2).fit_transform(X, y)[:5] # 链接：http://www.jianshu.com/p/2624521f87eb

array([[ 1.4,  0.2],
       [ 1.4,  0.2],
       [ 1.3,  0.2],
       [ 1.5,  0.2],
       [ 1.4,  0.2]])

In [23]:
import numpy as np
SelectKBest(lambda X, Y: tuple(map(tuple, np.array(list(map(lambda x:pearsonr(x, Y), X.T))).T)), k=2).fit_transform(X, y)[:5]

array([[ 1.4,  0.2],
       [ 1.4,  0.2],
       [ 1.3,  0.2],
       [ 1.5,  0.2],
       [ 1.4,  0.2]])

In [27]:
map(lambda x:pearsonr(x, y), X.T)

In [28]:
list(map(lambda x:pearsonr(x, y), X.T))

[(0.78256123181008141, 2.8904783526140542e-32),
 (-0.41944620026002749, 9.1599849725500019e-08),
 (0.94904254485233364, 4.1554775794971695e-76),
 (0.95646382380161732, 4.7750023687566194e-81)]

In [29]:
np.array(list(map(lambda x:pearsonr(x, y), X.T)))

array([[  7.82561232e-01,   2.89047835e-32],
       [ -4.19446200e-01,   9.15998497e-08],
       [  9.49042545e-01,   4.15547758e-76],
       [  9.56463824e-01,   4.77500237e-81]])

In [31]:
tuple(map(tuple, np.array(list(map(lambda x:pearsonr(x, y), X.T)))))

((0.78256123181008141, 2.8904783526140542e-32),
 (-0.41944620026002749, 9.1599849725500019e-08),
 (0.94904254485233364, 4.1554775794971695e-76),
 (0.95646382380161732, 4.7750023687566194e-81))

In [32]:
lambda X, Y: tuple(map(tuple, np.array(list(map(lambda x:pearsonr(x, Y), X.T))).T))

<function __main__.<lambda>>

### 卡方检验
经典的卡方检验是检验定性自变量对定性因变量的相关性。假设自变量有N种取值，因变量有M种取值，考虑自变量等于i且因变量等于j的样本频数的观察值与期望的差距，构建统计量：<img src='http://upload-images.jianshu.io/upload_images/1212395-668c2a5672629f47.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240'>
这个统计量的含义简而言之就是自变量对因变量的相关性。用feature_selection库的SelectKBest类结合卡方检验来选择特征的代码如下：

In [30]:
from sklearn.feature_selection import SelectKBest, chi2
#选择K个最好的特征，返回选择特征后的数据
SelectKBest(chi2, k=2).fit_transform(X,y)[:5]

array([[ 1.4,  0.2],
       [ 1.4,  0.2],
       [ 1.3,  0.2],
       [ 1.5,  0.2],
       [ 1.4,  0.2]])

### 互信息法
经典的互信息也是评价定性自变量对定性因变量的相关性的，互信息计算公式如下：
<img src='http://images2015.cnblogs.com/blog/927391/201605/927391-20160502145723247-1184422794.png'>
为了处理定量数据，最大信息系数法被提出，使用feature_selection库的SelectKBest类结合最大信息系数法来选择特征的代码如下

In [39]:
from sklearn.feature_selection import SelectKBest, mutual_info_classif

#选择K个最好的特征，返回特征选择后的数据
SelectKBest(mutual_info_classif, k=2).fit_transform(X, y)[:5]

array([[ 1.4,  0.2],
       [ 1.4,  0.2],
       [ 1.3,  0.2],
       [ 1.5,  0.2],
       [ 1.4,  0.2]])

互信息用于特征选择有以下缺点：
* 它不属于度量方式，也没有办法归一化，在不同数据集上的结果无法做比较；
* 对于连续变量的计算不是很方便（X和Y都是集合，x，y都是离散的取值），通常变量需要先离散化，而互信息的结果对离散化的方式很敏感。

最大信息系数（Maximal Information Coefficient， MIC）克服了这两个问题。它首先寻找一种最优的离散化方式，然后把互信息取值转换成一种度量方式，取值区间在[0，1]。minepy提供了MIC功能，代码如下：

In [ ]:
from minepy import MINE

m = MINE()
x = np.random.uniform(-1, 1, 10000)
m.compute_score(x, x**2)
print(m.mic())

## Wrapper
### 递归特征消除法
递归消除特征法使用一个基模型来进行多轮训练，每轮训练后，消除若干权值系数的特征，再基于新的特征集进行下一轮训练。使用feature_selection库的RFE类来选择特征的代码如下：

In [42]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
#递归特征消除法，返回特征选择后的数据
#参数estimator为基模型
#参数n_features_to_select为选择的特征个数
RFE(estimator=LogisticRegression(), n_features_to_select=2).fit_transform(X,y)[:5]

array([[ 3.5,  0.2],
       [ 3. ,  0.2],
       [ 3.2,  0.2],
       [ 3.1,  0.2],
       [ 3.6,  0.2]])

## Enbedded
### 基于惩罚项的特征选择法
使用带惩罚项的基模型，除了筛选出特征外，同时也进行了降维。使用feature_selection库的SelectFromModel类结合带L1惩罚项的逻辑回归模型，来选择特征的代码如下：

In [43]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
#带L1惩罚项的逻辑回归作为基模型的特征选择
SelectFromModel(LogisticRegression(penalty="l1", C=0.1)).fit_transform(X,y)[:5]

array([[ 5.1,  3.5,  1.4],
       [ 4.9,  3. ,  1.4],
       [ 4.7,  3.2,  1.3],
       [ 4.6,  3.1,  1.5],
       [ 5. ,  3.6,  1.4]])

　L1惩罚项降维的原理在于保留多个对目标值具有同等相关性的特征中的一个，所以没选到的特征不代表不重要。故，可结合L2惩罚项来优化。具体操作为：若一个特征在L1中的权值为1，选择在L2中权值差别不大且在L1中权值为0的特征构成同类集合，将这一集合中的特征平分L1中的权值，故需要构建一个新的逻辑回归模型：

In [44]:
from sklearn.linear_model import LogisticRegression

class LR(LogisticRegression):
    def __init__(self, threshold=0.01, dual=False, tol=1e-4, C=1.0,
                 fit_intercept=True, intercept_scaling=1, class_weight=None,
                 random_state=None, solver='liblinear', max_iter=100,
                 multi_class='ovr', verbose=0, warm_start=False, n_jobs=1):

        #权值相近的阈值
        self.threshold = threshold
        LogisticRegression.__init__(self, penalty='l1', dual=dual, tol=tol, C=C,
                 fit_intercept=fit_intercept, intercept_scaling=intercept_scaling, class_weight=class_weight,
                 random_state=random_state, solver=solver, max_iter=max_iter,
                 multi_class=multi_class, verbose=verbose, warm_start=warm_start, n_jobs=n_jobs)
        #使用同样的参数创建L2逻辑回归
        self.l2 = LogisticRegression(penalty='l2', dual=dual, tol=tol, C=C, fit_intercept=fit_intercept, intercept_scaling=intercept_scaling, class_weight = class_weight, random_state=random_state, solver=solver, max_iter=max_iter, multi_class=multi_class, verbose=verbose, warm_start=warm_start, n_jobs=n_jobs)

    def fit(self, X, y, sample_weight=None):
        #训练L1逻辑回归
        super(LR, self).fit(X, y, sample_weight=sample_weight)
        self.coef_old_ = self.coef_.copy()
        #训练L2逻辑回归
        self.l2.fit(X, y, sample_weight=sample_weight)

        cntOfRow, cntOfCol = self.coef_.shape
        #权值系数矩阵的行数对应目标值的种类数目
        for i in range(cntOfRow):
            for j in range(cntOfCol):
                coef = self.coef_[i][j]
                #L1逻辑回归的权值系数不为0
                if coef != 0:
                    idx = [j]
                    #对应在L2逻辑回归中的权值系数
                    coef1 = self.l2.coef_[i][j]
                    for k in range(cntOfCol):
                        coef2 = self.l2.coef_[i][k]
                        #在L2逻辑回归中，权值系数之差小于设定的阈值，且在L1中对应的权值为0
                        if abs(coef1-coef2) < self.threshold and j != k and self.coef_[i][k] == 0:
                            idx.append(k)
                    #计算这一类特征的权值系数均值
                    mean = coef / len(idx)
                    self.coef_[i][idx] = mean
        return self

使用feature_selection库的SelectFromModel类结合带L1以及L2惩罚项的逻辑回归模型，来选择特征的代码如下：

In [45]:
from sklearn.feature_selection import SelectFromModel
#带L1和L2惩罚项的逻辑回归作为基模型的特征选择
#参数threshold为权值系数之差的阈值
SelectFromModel(LR(threshold=0.5, C=0.1)).fit_transform(X, y)[:5]

array([[ 5.1,  3.5,  1.4,  0.2],
       [ 4.9,  3. ,  1.4,  0.2],
       [ 4.7,  3.2,  1.3,  0.2],
       [ 4.6,  3.1,  1.5,  0.2],
       [ 5. ,  3.6,  1.4,  0.2]])

### 基于树模型的特征选择法
树模型中GBDT也可用来作为基模型进行特征选择，使用feature_selection库的SelectFromModel类结合GBDT模型，来选择特征的代码如下：

In [46]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import GradientBoostingClassifier
#GBDT作为基模型的特征选择
SelectFromModel(GradientBoostingClassifier()).fit_transform(X,y)[:5]

array([[ 1.4,  0.2],
       [ 1.4,  0.2],
       [ 1.3,  0.2],
       [ 1.5,  0.2],
       [ 1.4,  0.2]])

In [9]:
iris.feature_names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [12]:
# 随机森林作为模型
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor().fit(X,y)
sorted(zip(rf.feature_importances_, iris.feature_names), reverse=True)

[(0.64385218121207954, 'petal length (cm)'),
 (0.3434120945801647, 'petal width (cm)'),
 (0.0073114179665177805, 'sepal length (cm)'),
 (0.0054243062412380697, 'sepal width (cm)')]

## 总结

| 类	| 所属方式 | 说明 |  
| :--- | :----- | :--- |
| VarianceThreshold	| Filter| 方差选择法| 
| SelectKBest	| Filter | 可选关联系数、卡方校验、最大信息系数作为得分计算的方法| 
| RFE	| Wrapper	| 递归地训练基模型，将权值系数较小的特征从特征集合中消除| 
| SelectFromModel	| Embedded	| 训练基模型，选择权值系数较高的特征| 

# 降维
当特征选择完成后，可以直接训练模型了，但是可能由于特征矩阵过大，导致计算量大，训练时间长的问题，因此降低特征矩阵维度也是必不可少的。常见的降维方法除了以上提到的基于L1惩罚项的模型以外，另外还有主成分分析法（PCA）和线性判别分析（LDA），线性判别分析本身也是一个分类模型。PCA和LDA有很多的相似点，其本质是要将原始的样本映射到维度更低的样本空间中，但是PCA和LDA的映射目标不一样：PCA是为了让映射后的样本具有最大的发散性；而LDA是为了让映射后的样本有最好的分类性能。所以说PCA是一种无监督的降维方法，而LDA是一种有监督的降维方法。

## PCA

In [47]:
from sklearn.decomposition import PCA
# 主成分分析法，返回降维后的数据
# 参数n_components为主成分个数
PCA(n_components=2).fit_transform(X,y)[:5]

array([[-2.68420713,  0.32660731],
       [-2.71539062, -0.16955685],
       [-2.88981954, -0.13734561],
       [-2.7464372 , -0.31112432],
       [-2.72859298,  0.33392456]])

## LDA
sklearn.discriminant_analysis.LinearDiscriminantAnalysis(solver=’svd’, shrinkage=None, priors=None, n_components=None, store_covariance=False, tol=0.0001)

In [6]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# 线性判别分析法，返回降维后的数据
LinearDiscriminantAnalysis(n_components=2).fit_transform(X,y)[:5]

array([[-8.0849532 ,  0.32845422],
       [-7.1471629 , -0.75547326],
       [-7.51137789, -0.23807832],
       [-6.83767561, -0.64288476],
       [-8.15781367,  0.54063935]])

## 总结
| 库 | 类	| 说明| 
| :---| :---| :----| 
| decomposition	| PCA	| 主成分分析法| 
| lda	| LDA	| 线性判别分析法| 

<img src='http://images2015.cnblogs.com/blog/927391/201604/927391-20160430145122660-830141495.jpg'>